In [1]:
import pandas as pd 
from sqlalchemy import create_engine
server = 'rice-sqlcl-db'
database = 'SharadarEquity'
username = 'stocks'
password = '6LAZH1'
string = f"mssql+pymssql://{username}:{password}@{server}/{database}"
conn = create_engine(string).connect() 
df = pd.read_sql("select * from ghz", conn)
df = df.dropna(subset=["date"])

In [2]:
prices =pd.read_sql("select * from prices where date >= '2020-01-01' and date < '2025-04-16'", conn)
metrics = pd.read_sql("select * from metrics where date >= '2020-01-01' and date < '2025-04-16'", conn)
tickers = pd.read_sql("select * from tickers", conn)

In [3]:
df = prices.merge(metrics, left_on = ["date", "ticker"], right_on=["date", "ticker"], how="inner")
df = df.merge(tickers[["ticker", "industry", "sector"]], left_on="ticker", right_on="ticker", how="inner")
df = df.drop(columns=["lastupdated_x", "lastupdated_y"])
df = df.rename(columns={"opn": "open"})
df = df.sort_values(by=["ticker", "date"])
df = df.set_index(["ticker", "date"])

In [4]:
df["lag52"] = df.groupby("ticker", group_keys=False).closeadj.apply(lambda x: x/x.shift(52)-1)
df["lag26"] = df.groupby("ticker", group_keys=False).closeadj.apply(lambda x: x/x.shift(26)-1)
df["lag4"] = df.groupby("ticker", group_keys=False).closeadj.apply(lambda x: x/x.shift(4)-1)
df["lag1"] = df.groupby("ticker", group_keys=False).closeadj.apply(lambda x: x/x.shift(1)-1)
df["ret"] = df.groupby("ticker", group_keys=False).closeadj.apply(lambda x: x.shift(-1)/x-1)
df = df.reset_index()

In [5]:

df2 = df[df.date.astype(str)=="2021-01-01"].copy()
df2 = df2.dropna()
df2["rnk"] = df2.marketcap.rank(ascending=False)
df3 = df2[(df2.rnk>1000) & (df2.rnk<=2000)]
tick_lst = df3.ticker.to_list()
df2 = df[(df.ticker.isin(tick_lst)) & (df.date.astype(str)>="2021-01-01")].dropna()
df2.to_csv("stocks.csv")